In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import resnet18, ResNet18_Weights
import matplotlib.pyplot as plt
import time
import copy
import os
import cv2
import torch
from PIL import Image
import torchvision.transforms as transforms # not to be confused with tensorflow
from torchvision.transforms.functional import crop
from io import BytesIO
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, random_split

from google.colab import drive
drive.mount('/content/drive')

torch.manual_seed(42)
torch.cuda.manual_seed(42)

Mounted at /content/drive


In [3]:
path = '/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/radset.pt'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = torch.load(path)
train_length = int(0.9 * len(data))
train_data, test_data = torch.utils.data.random_split(data, [train_length, len(data) - train_length])
print(train_length, len(data) - train_length)

1296 144


In [4]:
# using resnet50 from the pytorch library
#model.fc.register_forward_hook(lambda m, inp, out: F.dropout(out, p=0.5, training=m.training))

val_acc = []
val_loss = []

trainbsize = 8
testbsize = 8

training_acc = []
trainin_loss = []
train_loader = DataLoader(train_data, batch_size = trainbsize, shuffle = True, pin_memory = True, num_workers = 2)
test_loader = DataLoader(test_data, batch_size = testbsize, shuffle = False, pin_memory = True, num_workers = 2)

def preprocess_again(image, label, batch):
  label = label.squeeze(1)
  return image, label

In [5]:
epochs = 10
model = resnet18(pretrained = True).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.0005, momentum = 0.9, weight_decay = 1e-5)

val_acc = []
val_loss = []
training_acc = []
training_loss = []

for epoch in range(1, epochs+1):
  model.train()
  train_loss = 0.00
  correct = 0.00
  total = 0.00
  for image, label in train_loader:
    image, label = preprocess_again(image, label, trainbsize)
    image = image.cuda()
    label = label.cuda()
    optimizer.zero_grad()
    y_predict = model(image).to(device)
    loss = criterion(y_predict, label)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()
    _, prediction = y_predict.max(dim=1)
    correct += (prediction.eq(label).sum().item())
    total += label.size(0)
  training_acc.append(correct / total)
  training_loss.append(train_loss/len(train_loader))
  print(f"Epoch: [{epoch}/{epochs}], Training Accuracy: {training_acc[epoch - 1]:.4f}, Training Loss: {training_loss[epoch - 1]:.4f}")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 192MB/s]


Epoch: [1/10], Training Accuracy: 0.4182, Training Loss: 2.3834
Epoch: [2/10], Training Accuracy: 0.7531, Training Loss: 0.7441
Epoch: [3/10], Training Accuracy: 0.8827, Training Loss: 0.3810
Epoch: [4/10], Training Accuracy: 0.9514, Training Loss: 0.2073
Epoch: [5/10], Training Accuracy: 0.9606, Training Loss: 0.1470
Epoch: [6/10], Training Accuracy: 0.9838, Training Loss: 0.0885
Epoch: [7/10], Training Accuracy: 0.9907, Training Loss: 0.0476
Epoch: [8/10], Training Accuracy: 0.9884, Training Loss: 0.0584
Epoch: [9/10], Training Accuracy: 0.9900, Training Loss: 0.0544
Epoch: [10/10], Training Accuracy: 0.9869, Training Loss: 0.0558


In [6]:
model.eval()
correct = 0.00
total = 0.00
_val_loss = 0.00

with torch.no_grad():
  for image, label in test_loader:
    image, label = preprocess_again(image, label, testbsize)
    image = image.cuda()
    label = label.cuda()
    y_pred = model(image).to(device)
    _, prediction = y_pred.max(dim=1)
    correct += prediction.eq(label).sum().item()
    total += label.size(0)

print(f"Validation Accuracy: {correct/total:4f}")

Validation Accuracy: 0.812500


In [7]:
torch.save(model, "/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/amodel.pt")